In [218]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [320]:
import pandas as pd
import os

In [321]:
def parse_usages(usage_string):
    usages = []
    usage_string = usage_string.replace("),C",")\tC")
    usage_string = usage_string.split("\t")
    usages_pattern = r"[A-Za-z]PU: \([^)]*\)"
    cpu_usage_pattern = r"maxrss: (\d+MiB)"
    gpu_usage_pattern = r"U: (\d+MiB)"
    mapper = {
        0: "before (MiB)",
        1: "after (MiB)"
    }
    for i, moments in enumerate(usage_string):
            matches = re.findall(usages_pattern, moments)
            for match in matches:
                match = match.replace(",","")
                cpu = re.findall(cpu_usage_pattern, match)
                gpu = re.findall(gpu_usage_pattern, match)
                if match[:3] == "CPU":
                    usages.append({f"{match[:3]}-{mapper[i]}":int(cpu[0].replace("MiB",""))})
                elif match[:3] == "GPU":
                    usages.append({f"{match[:3]}-{mapper[i]}":int(gpu[0].replace("MiB",""))})                    
    return usages
            
        

In [322]:
def parse_file(file):
    lazy_load = True if "--lazy-load" in file else False
    vllm_tensorized = True if "--plaid-mode"in file else False
    df = pd.read_csv(file)
    rows = []
    for i in range(len(df)):
        row = df.iloc[i]
        load_format = row.name[0]
        model = row.name[1]
        duration = row[-2]
        cache = row[-1]
        to_concat = row.name[2:]
        mem_before_after = ",".join(to_concat)
        vals = {}
        vals.update({"load_format":load_format})
        vals.update({"model":model})
        for dic in parse_usages(mem_before_after):
            vals.update(dic)
        vals.update({"duration(s)":duration})
        vals.update({"cache_hit":cache})
        vals.update({"lazy_load":lazy_load})
        vals.update({"vllm_tensorized":vllm_tensorized})
        cpu_diff = vals["CPU-after (MiB)"] - vals["CPU-before (MiB)"]
        gpu_diff = vals["GPU-after (MiB)"] - vals["GPU-before (MiB)"]
        vals.update({"CPU-diff-MiB": cpu_diff})
        vals.update({"GPU-diff-MiB": gpu_diff})
        rows.append(vals)
    return pd.DataFrame(rows)


In [323]:
files = [file for file in os.listdir() if file.isnumeric()]
dfs = []
for file in files:
    load_type = os.listdir(file)[0]
    model_ref_1 = os.listdir(f"{file}/{load_type}")[0]
    model_ref_2 = os.listdir(f"{file}/{load_type}/{model_ref_1}")[0]
    localities = os.listdir(f"{file}/{load_type}/{model_ref_1}/{model_ref_2}")
    for locality in localities:
        subpath = f"{file}/{load_type}/{model_ref_1}/{model_ref_2}/{locality}"
        benchmark_files = os.listdir(subpath)
        for benchmark in benchmark_files:
            created_df = parse_file(f"{subpath}/{benchmark}")
            created_df["local_or_not_local"] = locality
            dfs.append(created_df)
df = pd.concat(dfs)
        

/tmp/ipykernel_106143/3683185005.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  duration = row[-2]
/tmp/ipykernel_106143/3683185005.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cache = row[-1]
/tmp/ipykernel_106143/3683185005.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  duration = row[-2]
/tmp/ipykernel_106143/3683185005.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a fut

In [324]:
#df['cache_hit'] = df['cache_hit'].fillna("not_tensorizer")

In [325]:
mask = ((df['load_format']=="tensorizer") & (df['vllm_tensorized'] == False))

In [326]:
test_df = df[~mask]

In [327]:
test_df.groupby(by=["load_format", "model", "cache_hit"])[['duration(s)']].mean()

duration(s)
load_format model                  cache_hit             
tensorizer  EleutherAI/pythia-1.4b HIT          11.816452
                                   MISS         12.877589

In [328]:
df

,load_format,model,CPU-before (MiB),GPU-before (MiB),CPU-after (MiB),GPU-after (MiB),duration(s),cache_hit,lazy_load,vllm_tensorized,CPU-diff-MiB,GPU-diff-MiB,local_or_not_local
0,npcache,meta-llama/Llama-2-13b-hf,995,262,21173,44654,56.318969,NaN,False,False,20178,44392,non-local
1,npcache,meta-llama/Llama-2-13b-hf,995,262,21170,44654,71.559784,NaN,False,False,20175,44392,non-local
2,npcache,meta-llama/Llama-2-13b-hf,995,262,21170,44654,65.067494,NaN,False,False,20175,44392,non-local
3,npcache,meta-llama/Llama-2-13b-hf,995,262,17617,44654,71.059357,NaN,False,False,16622,44392,non-local
4,npcache,meta-llama/Llama-2-13b-hf,995,262,21171,44654,60.608334,NaN,False,False,20176,44392,non-local
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,safetensors,meta-llama/Llama-2-13b-hf,997,262,11728,44654,35.300731,NaN,False,False,10731,44392,non-local
126,safetensors,meta-llama/Llama-2-13b-hf,997,262,11728,44654,61.003674,NaN,False,False,10731,44392,non-local
127,safetensors,meta-llama/Llama-2-13b-hf,996,262,11728,44654,55.122638,NaN,False,False,10732,44392,non-local
128,safetensors,meta-llama/Llama-2-13b-hf,997,262,11728,44654,36.169756,NaN,False,False,10731,44392,non-local


In [334]:
df[(df["model"] == "meta-llama/Llama-2-13b-hf") & (df["load_format"] == "tensorizer")]

,load_format,model,CPU-before (MiB),GPU-before (MiB),CPU-after (MiB),GPU-after (MiB),duration(s),cache_hit,lazy_load,vllm_tensorized,CPU-diff-MiB,GPU-diff-MiB,local_or_not_local
0,tensorizer,meta-llama/Llama-2-13b-hf,996,262,27032,44654,53.442105,HIT,True,False,26036,44392,non-local
1,tensorizer,meta-llama/Llama-2-13b-hf,996,262,27030,44654,62.241688,HIT,True,False,26034,44392,non-local
2,tensorizer,meta-llama/Llama-2-13b-hf,997,262,27030,44654,72.614542,HIT,True,False,26033,44392,non-local
3,tensorizer,meta-llama/Llama-2-13b-hf,996,262,27029,44654,51.446487,HIT,True,False,26033,44392,non-local
4,tensorizer,meta-llama/Llama-2-13b-hf,996,262,27029,44654,53.152895,HIT,True,False,26033,44392,non-local
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,tensorizer,meta-llama/Llama-2-13b-hf,995,262,27026,44654,66.294306,HIT,False,False,26031,44392,non-local
126,tensorizer,meta-llama/Llama-2-13b-hf,995,262,27026,44654,50.059149,HIT,False,False,26031,44392,non-local
127,tensorizer,meta-llama/Llama-2-13b-hf,995,262,27026,44654,52.439728,HIT,False,False,26031,44392,non-local
128,tensorizer,meta-llama/Llama-2-13b-hf,995,262,27027,44654,58.328505,HIT,False,False,26032,44392,non-local
